### **1. Conectar con la Base de Datos**                                        
Se detallan los parametros para realizar la conexión:                            

*   **Host:** Dirección del servidor de la base de datos
*   **Puerto:** Puerto 5432
*   Nombre de la base de datos
*   Usuario
*   Contraseña

In [ ]:
import psycopg2
import pandas as pd

# Parámetros de conexión a PostgreSQL
host = 'colocar nombre de host'
port = '5432'  # Puerto por defecto
dbname = 'rsm'
user = 'postgres'
password = 'colocar constraseña'

# Establecer la conexión
conn = psycopg2.connect(host=host, port=port, dbname=dbname, user=user, password=password)

# Crear un cursor para ejecutar consultas SQL
cursor = conn.cursor()


### **2. Código para obetner los datos SQL**

In [ ]:
# Consulta SQL para obtener datos de ventas, productos y clientes
query = """
    SELECT v.VentaID, v.ClienteID, v.ProductoID, v.Cantidad, v.FechaVenta, v.Region, v.VentaTotal,
           c.NombreCliente, c.Email, p.NombreProducto, p.PrecioUnitario
    FROM Ventas v
    JOIN Clientes c ON v.ClienteID = c.ClienteID
    JOIN Productos p ON v.ProductoID = p.ProductoID
    ORDER BY v.FechaVenta;
"""

# Ejecutar la consulta y cargar los resultados en un DataFrame
ventas_df = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame
ventas_df.head()


### **3. Identificación de Patrones, Tendencias y Anomalías**

**3.1 Código para análisis de series temporales (ventas por mes)**

In [ ]:
# Convertir la columna FechaVenta a formato datetime
ventas_df['FechaVenta'] = pd.to_datetime(ventas_df['FechaVenta'])

# Agrupar las ventas por mes
ventas_df['Mes'] = ventas_df['FechaVenta'].dt.to_period('M')
ventas_mensuales = ventas_df.groupby('Mes')['VentaTotal'].sum().reset_index()

# Visualizar las tendencias de ventas mensuales
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(ventas_mensuales['Mes'].astype(str), ventas_mensuales['VentaTotal'], marker='o', color='b')
plt.title('Ventas Totales por Mes')
plt.xlabel('Mes')
plt.ylabel('Ventas Totales')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

**3.2 Código para clasificar clientes según frecuencia de compra**

In [ ]:
# Segmentación de Clientes: Gasto Total por Cliente
clientes_segmentados = ventas_df.groupby('ClienteID').agg(
    TotalCompras=('VentaTotal', 'count'),
    GastoTotal=('VentaTotal', 'sum'),
    GastoPromedio=('VentaTotal', 'mean')
).reset_index()

# Visualizar el Top 10 clientes con mayor gasto total
plt.figure(figsize=(10, 6))
clientes_segmentados_sorted = clientes_segmentados.sort_values(by='GastoTotal', ascending=False).head(10)
plt.bar(clientes_segmentados_sorted['ClienteID'], clientes_segmentados_sorted['GastoTotal'], color='orange')
plt.title('Top 10 Clientes con Mayor Gasto Total')
plt.xlabel('Cliente ID')
plt.ylabel('Gasto Total')
plt.xticks(rotation=45)
plt.show()

**3.3 Código para identificar picos de ventas**

In [ ]:
# Identificación de Anomalías
mean_sales = ventas_df['VentaTotal'].mean()
std_sales = ventas_df['VentaTotal'].std()

# Detectamos las anomalías (ventas fuera de 2 desviaciones estándar)
ventas_df['Anomalia'] = (ventas_df['VentaTotal'] > mean_sales + 2 * std_sales) | (ventas_df['VentaTotal'] < mean_sales - 2 * std_sales)
anomalies = ventas_df[ventas_df['Anomalia'] == True]

# Visualizar las ventas diarias con anomalías
plt.figure(figsize=(10, 6))
plt.plot(ventas_df['FechaVenta'], ventas_df['VentaTotal'], label='Ventas Diarias')
plt.scatter(anomalies['FechaVenta'], anomalies['VentaTotal'], color='r', label='Anomalías', zorder=5)
plt.title('Ventas Diarias con Anomalías')
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.legend()
plt.xticks(rotation=45)
plt.show()

### **4. Cerrar la Conexión a la Base de Datos**

In [ ]:
# Cerrar la conexión
cursor.close()
conn.close()